In [1]:
import fasttext
import fasttext.util
import numpy as np
import heapq
import pandas as pd
from scipy.optimize import linear_sum_assignment
import random

# Load FastText embeddings
fasttext.util.download_model('en', if_exists='ignore')  # English model
en_model = fasttext.load_model('cc.en.300.bin')

In [2]:
fasttext.util.reduce_model(en_model, 100)

In [3]:
import json
from pprint import pprint
from functools import lru_cache
from nltk.stem import PorterStemmer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
# import Levenshtein

In [4]:
stemmer = PorterStemmer()
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tomi_owolabi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tomi_owolabi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
SIM_VALUES = "similarity_data/sim.values.txt"
SIM_WORDS = "similarity_data/sim.words.txt"

In [123]:
class WordDistance():
    def __init__(self, emb_model, words_path=None, sim_path=None):
        self.model = emb_model
        self.words = []
        self.sim_values = []
        if words_path and sim_path: 
            with open(words_path, "r") as f:
                self.words = f.readlines()
            with open(sim_path, "r") as f:
                self.sim_values = f.readlines()
        self.num_words = len(self.words)
    
    def embed_word(self, word):
        """Get the FastText embedding for a word."""
        return self.model.get_word_vector(word)
    
    @staticmethod
    def cosine_distance(vec1, vec2):
        """Compute cosine distance between two vectors."""
        return 1 - np.dot(vec1, vec2) / ((np.linalg.norm(vec1) * np.linalg.norm(vec2)) + 1e-8) #numerical stability
    
    @lru_cache(maxsize=1024)
    def _word_cosine_distance(self, word1, word2):
        word1_emb = self.embed_word(word1)
        word2_emb = self.embed_word(word2)
        return round(self.cosine_distance(word1_emb, word2_emb), 6)
    
    def c_word_cosine_distance(self, word1, word2):
        # return random.randint(1, 10)/10
        try:
            w1_index = self.words.index(word1)
            w2_index = self.words.index(word2)
            cache_index = w1_index * self.num_words + w2_index
            return self.sim_values[cache_index]
        except ValueError:
            return self._word_cosine_distance(word1, word2)

In [130]:
def load_word_data(words_path=None, sim_path=None):
    """Load word data and similarity values."""
    words = []
    sim_values = []
    if words_path and sim_path:
        with open(words_path, "r") as f:
            words = [x.strip() for x in f.readlines()]
        with open(sim_path, "r") as f:
            sim_values = [float(x) for x in f.readlines()]
    return words, sim_values

In [ ]:
__model = en_model
__words, __sim_values = load_word_data("similarity_data/sim.words.txt", "similarity_data/sim.values.txt")
__num_words = len(__words)
__word2idx = {word: idx for word, idx in zip(__words, range(len(__words)))}

__hits = 0
__misses = 0

def embed_word(word):
    """Get the FastText embedding for a word."""
    return __model.get_word_vector(word)

def cosine_distance(vec1, vec2):
    """Compute cosine distance between two vectors."""
    return 1 - np.dot(vec1, vec2) / ((np.linalg.norm(vec1) * np.linalg.norm(vec2)) + 1e-8)  # numerical stability

@lru_cache(maxsize=1024)
def word_cosine_distance(word1, word2):
    """Compute cosine distance between two words."""
    word1_emb = embed_word(word1)
    word2_emb = embed_word(word2)
    return cosine_distance(word1_emb, word2_emb)
    # return int(1000 * cosine_distance(word1_emb, word2_emb))

@lru_cache(maxsize=10000)
def get_stem(word):
    return stemmer.stem(word)

@lru_cache(maxsize=2048)
def cached_word_cosine_distance(word1, word2):
    # return 0
    """Compute cached cosine distance between two words."""
    global __hits, __misses
    w1_index = __word2idx.get(word1, None)
    w2_index = __word2idx.get(word2, None)
    if w1_index and w2_index:
        __hits += 1
        cache_index = w1_index * __num_words + w2_index
        # return int(1000 * __sim_values[cache_index])
        return __sim_values[cache_index]

    else:
        __misses += 1
        # return 0
        return word_cosine_distance(word1, word2)

In [44]:
# len(__words) * len(__words)
len(__sim_values)

45927729

In [9]:
def clean_text(text):
    """
    Removes stop words and punctuation from the given text.

    Args:
        text (str): The input text.

    Returns:
        str: The cleaned text.
    """
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Get the stop words and punctuation
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    
    # Filter out stop words and punctuation
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words and word not in punctuation]
    
    # Join the tokens back into a string
    return ' '.join(filtered_tokens)

In [10]:
import zipfile

def read_file_from_zip(zip_path, file_name):
    """
    Reads the content of a specific file from a zipped folder without extracting the entire folder.

    Args:
        zip_path (str): Path to the zip file.
        file_name (str): Name of the file to read within the zip.

    Returns:
        str: Content of the file as a string.
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        with zip_ref.open(file_name) as file:
            return file.read().decode('utf-8')

In [11]:
def get_knowledge_store_for_claim(claim_id):
    references = []
    try:
        claim_file = read_file_from_zip("baseline/AVeriTeC/data_store/knowledge_store/dev_knowledge_store.zip", f"output_dev/{claim_id}.json")
    except Exception as e:
        print(e)
        return None

    for line in claim_file.splitlines():
        try:
            this_ref = json.loads(line)
            references.append(this_ref)
        except Exception as e:
            print(e)
            continue
    return references

In [46]:
def optimal_matching(query_embeddings, text_embeddings):
    """Find an optimal matching between query words and text words using the Hungarian algorithm."""
    cost_matrix = np.zeros((len(query_embeddings), len(text_embeddings)))
    
    for i, q_emb in enumerate(query_embeddings):
        for j, t_emb in enumerate(text_embeddings):
            cost_matrix[i, j] = cosine_distance(q_emb, t_emb)
    
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    total_score = -cost_matrix[row_ind, col_ind].sum()
    
    return total_score


cost_matrix = None #avoid having to assign memory with each call. Will resize cost matrix when shape changes
def word_optimal_matching(query_words, text_words):
    # cost_matrix = np.zeros((len(query_words), len(text_words)))
    # return 0.1
    global cost_matrix
    shape = (len(query_words), len(text_words))
    if cost_matrix is None or cost_matrix.shape != shape:
        cost_matrix = np.zeros(shape)
    else:
        cost_matrix[:] = 1
    for i, word1 in enumerate(query_words):
        for j, word2 in enumerate(text_words):
            cost_matrix[i, j] = cached_word_cosine_distance(word1, word2)
            
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    total_score = -cost_matrix[row_ind, col_ind].sum()
    # lev_score = Levenshtein.ratio(col_ind, sorted(col_ind))
    return total_score

def find_top_n_matches(query_text, target_text, patch_size=50, overlap=25, top_n=5):
    """Find the top N scoring spans in the target text using a sliding window without redundant embeddings."""
    target_words = [get_stem(word) for word in target_text.split()]
    query_words = [get_stem(word) for word in query_text.split()]
    heap = []    
    for start in range(0, max(1, len(target_words) - patch_size + 1), patch_size - overlap):
        end = start + patch_size
        curr_target_words = target_words[start:end]
        score = word_optimal_matching(query_words, curr_target_words)
        heapq.heappush(heap, (score, start, end))
        if len(heap) > top_n:
            heapq.heappop(heap)
    return sorted(heap, reverse=True)

In [13]:
dev_tasks = None
with open("/Users/tomi_owolabi/projects/cpsc601/baseline/AVeriTeC/data/dev.json") as f:
    dev_tasks = json.load(f)

In [14]:
def get_claim_by_id(claim_list, claim_id):
    filtered = claim_list[claim_id:claim_id+1]
    return filtered[0] if filtered else None

In [15]:
def get_span_text_from_claim_doc(claim_doc, span): 
    span = span[-2:]
    claim_text = " ".join(claim_doc.get("url2text")).split()
    return " ".join(claim_text[span[0]: span[1]])

In [112]:
def filter_claim_doc(claim, claim_doc_dict, patch_size):
    claim_text = " ".join(claim_doc_dict.get("url2text")).lower()[:3000]
    top_matches = find_top_n_matches(claim, claim_text, patch_size=patch_size, overlap=0)
    # claim_doc_dict["most_rel"] =[]
    # for i, match in enumerate(top_matches):
    #     # match_text = get_span_text_from_claim_doc(claim_doc_dict, match)
    #     claim_doc_dict["most_rel"].append(match)
    return claim_doc_dict    

In [17]:
# claim_id = 4
# claim = get_claim_by_id(dev_tasks, claim_id)
# pprint(claim.get("label"))
# claim = claim.get("claim")
# claim_knowledge = get_knowledge_store_for_claim(claim_id)
# print(len(claim_knowledge))
# claim_knowledge = filter_claim_doc(claim, claim_knowledge[5])

In [110]:
def process_claim(claim_list, claim_id):
    claim_id = claim_id
    claim = get_claim_by_id(claim_list, claim_id)
    claim_text = claim.get("claim", "").lower()
    patch_size = 64
    claim_docs = get_knowledge_store_for_claim(claim_id)
    claim_docs = [
        filter_claim_doc(claim_text, doc, patch_size) for doc in claim_docs[:]
    ]
    return claim_docs

In [118]:
print(f"calls: {__hits + __misses}")
print(f"hits: {__hits}, {(__hits/(__hits+__misses+1)):.2f}%")
print(f"misses: {__misses}, {(__misses/(__hits+__misses+1)):.2f}%")

calls: 4906722
hits: 1989473, 0.41%
misses: 2917249, 0.59%


In [ ]:
for i in range(20):
    print(len(get_knowledge_store_for_claim(i)))

In [134]:
import time
start_time = time.time()
g = process_claim(dev_tasks, 2)
end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")


Unterminated string starting at: line 1 column 786449 (char 786448)
Extra data: line 1 column 6 (char 5)
Extra data: line 1 column 6 (char 5)
Expecting value: line 1 column 1 (char 0)
Extra data: line 1 column 6 (char 5)
Extra data: line 1 column 3 (char 2)
Unterminated string starting at: line 1 column 786633 (char 786632)
Expecting value: line 1 column 1 (char 0)
Extra data: line 1 column 6 (char 5)
Extra data: line 1 column 3 (char 2)
Execution time: 22.78 seconds


In [44]:
# Example Usage
target_text = "This is a robust graph matching algorithm for string search. We apply it to find patterns in text efficiently."
query_text = "graph matching algorithm in bipartite graphs"

top_matches = find_top_n_matches(query_text, target_text)
for score, start, end in top_matches:
    print(f"Match score: {score:.4f}, Span: {' '.join(target_text.split()[start:end])}")

Match score: -1.0950, Span: This is a robust graph matching algorithm for string search. We apply it to find patterns in text efficiently.


In [37]:
N = 10000
words = pd.read_csv("unigram_freq.csv", dtype={"word": 'object'}, keep_default_na=False, na_values=[])["word"]
words = [x.lower() for x in words]
words = words[:N]
words = [stemmer.stem(word) for word in words]
words = sorted(list(set(words)))

similarity_index = [-1] * (len(words) * len(words))
for idx, word in enumerate(words):
    for idx2, word2 in enumerate(words):
        similarity_index[idx * len(words) + idx2] = word_cosine_distance(word, word2)

with open("similarity_data/sim.values.txt", "w") as f:
    for sim in similarity_index:
        f.write(f"{sim:2f}\n")

with open("similarity_data/sim.words.txt", "w") as f: 
    for word in words:
        f.write(f"{word}\n")

In [31]:
# similarity_index = None
# with open("similarity_index.txt", "r") as f:
#     similarity_index = [float(x) for x in f.readlines()]

# similarity_dict = {}
# for idx, word in enumerate(words):
#     for idx2, word2 in enumerate(words):
#         similarity_dict[f"{word}#{word2}"] = similarity_index[idx * len(words) + idx2]

In [81]:
__words

['a',
 'aa',
 'aaa',
 'aaron',
 'ab',
 'abandon',
 'abc',
 'aberdeen',
 'abil',
 'abl',
 'aborigin',
 'abort',
 'about',
 'abov',
 'abraham',
 'abroad',
 'absenc',
 'absent',
 'absolut',
 'absorpt',
 'abstract',
 'abu',
 'abus',
 'ac',
 'academ',
 'academi',
 'acc',
 'accent',
 'accept',
 'access',
 'accessori',
 'accid',
 'accommod',
 'accompani',
 'accomplish',
 'accord',
 'accordingli',
 'account',
 'accredit',
 'accur',
 'accuraci',
 'accus',
 'acdbent',
 'ace',
 'acer',
 'achiev',
 'acid',
 'acknowledg',
 'acm',
 'acn',
 'acoust',
 'acquir',
 'acquisit',
 'acr',
 'acrobat',
 'across',
 'acryl',
 'act',
 'action',
 'activ',
 'activist',
 'actor',
 'actress',
 'actual',
 'acut',
 'ad',
 'ada',
 'adam',
 'adapt',
 'adaptor',
 'add',
 'addict',
 'addit',
 'address',
 'adelaid',
 'adequ',
 'adida',
 'adipex',
 'adjac',
 'adjust',
 'admin',
 'administ',
 'administr',
 'admiss',
 'admit',
 'adob',
 'adolesc',
 'adopt',
 'adrian',
 'adsl',
 'adult',
 'advanc',
 'advantag',
 'adventur',
 '

In [133]:
__sim_values

[0.0,
 0.550554,
 0.744979,
 0.848233,
 0.740294,
 0.753646,
 0.872898,
 0.947873,
 0.71445,
 0.725884,
 1.016495,
 0.82227,
 0.640478,
 0.858973,
 0.856821,
 0.972242,
 1.032808,
 0.863104,
 0.717235,
 0.986695,
 0.859122,
 0.823554,
 0.862805,
 0.762871,
 0.816648,
 0.825239,
 0.780274,
 0.880491,
 0.731825,
 0.783766,
 0.834516,
 0.94247,
 0.843915,
 0.835194,
 0.8246,
 0.879746,
 1.049659,
 0.762097,
 0.837168,
 0.938754,
 0.980094,
 0.774667,
 0.889839,
 0.74154,
 0.829268,
 0.841474,
 0.924383,
 0.816601,
 0.863822,
 0.773569,
 1.15509,
 0.757455,
 1.07421,
 0.813577,
 0.836149,
 0.840244,
 0.916852,
 0.784471,
 0.827206,
 0.89532,
 0.841351,
 0.840952,
 0.88747,
 0.72109,
 0.88421,
 0.785367,
 0.722857,
 0.85984,
 0.844101,
 0.857286,
 0.636309,
 0.863392,
 0.82921,
 0.761535,
 0.915212,
 0.994642,
 1.009633,
 0.933755,
 1.159539,
 0.769882,
 0.989255,
 0.763435,
 0.799286,
 0.925864,
 0.751202,
 1.002029,
 1.071777,
 0.789124,
 0.919965,
 0.837641,
 0.791425,
 0.869882,
 0.8272